<img src='../../img/anaconda-logo.png' align='left' style="padding:10px">
<br>
*Copyright Continuum 2012-2016 All Rights Reserved.*

# Taxi rides Exercise

Make datashader plots for the following information by pickup location
* Rides with 5 or more passengers
* Color map all rides by time of day
  * can be by hour or bins of hours

Note: some these operations can be done in Pandas or after aggregation using xarray.

# Table of Contents
* [Taxi rides Exercise](#Taxi-rides-Exercise)
	* [Set-Up](#Set-Up)
	* [Data](#Data)
	* [Colors](#Colors)
	* [Projections](#Projections)
	* [By passenger count](#By-passenger-count)
		* [pandas](#pandas)
		* [xarray](#xarray)
	* [By time of day](#By-time-of-day)


## Set-Up

In [ ]:
import pandas as pd
import numpy as np
import datashader as ds
from datashader import transfer_functions as tf

## Data

In [ ]:
taxi = pd.read_csv('data/April1.csv')
taxi.head()

## Colors

In [ ]:
from datashader.colors import Greys9, inferno
Greys9_r = list(reversed(Greys9))[:-2]

## Projections

In [ ]:
from pyproj import Proj, transform
inProj = Proj(init='epsg:4326')
outProj = Proj(init='epsg:3857')

taxi['pickup_x'], taxi['pickup_y'] = \
  transform(inProj, outProj, taxi['pickup_longitude'].values, taxi['pickup_latitude'].values)

In [ ]:
x_range=(-8250000,-8210000)
y_range=(4965000,4990000)

## By passenger count

### pandas

In [ ]:
many_passengers = taxi.loc[taxi['passenger_count'] >= 5]

In [ ]:
cvs = ds.Canvas(plot_width=800, plot_height=500, x_range=x_range, y_range=y_range)
agg = cvs.points(many_passengers, 'pickup_x', 'pickup_y')
tf.shade(agg, cmap=reversed(inferno), how='eq_hist')

### xarray

In [ ]:
taxi['passenger_cat'] = taxi['passenger_count'].astype('category')

In [ ]:
cvs = ds.Canvas(plot_width=800, plot_height=500, x_range=x_range, y_range=y_range)
agg = cvs.points(taxi, 'pickup_x', 'pickup_y', ds.count_cat('passenger_cat'))
tf.shade(agg.sel(passenger_cat=[5,6,7]).sum(dim='passenger_cat'), cmap=reversed(inferno), how='eq_hist')

## By time of day

In [ ]:
taxi['hour'] = pd.to_datetime(taxi['tpep_pickup_datetime']).dt.hour
taxi['time'] = pd.cut(taxi['hour'], bins=[0, 12, 17, 24], labels=['morning', 'afternoon', 'evening']).astype('category')

In [ ]:
colors = {'morning':'blue', 'afternoon':'yellow', 'evening':'red'}

cvs = ds.Canvas(plot_width=800, plot_height=500, x_range=x_range, y_range=y_range)
agg = cvs.points(taxi, 'pickup_x', 'pickup_y', ds.count_cat('time'))
tf.shade(agg, color_key=colors, how='eq_hist')

---
*Copyright Continuum 2012-2016 All Rights Reserved.*